This program can be used to generate burst transactions that replicate fraudulent transactions in financial transactions. The burst transactions are inserted at every "prime" row.

In [1]:
import csv
import math

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def split_debit_transactions(input_file, output_file):
    with open(input_file, 'r') as csv_file:
        reader = csv.DictReader(csv_file)
        fieldnames = ['transaction_id', 'credit_amt', 'debit_amt', 'balance', 'date', 'transaction_type']
        transactions = list(reader)

    new_transactions = []
    transaction_id = 1

    for transaction in transactions:
        if transaction['transaction_type'] == 'debit':
            debit_amt = float(transaction['debit_amt'])
            balance = float(transaction['balance'])
            num_splits = 2
            while not is_prime(num_splits):
                num_splits += 1

            split_amount = debit_amt / num_splits

            for i in range(num_splits):
                new_transaction = {
                   
                    'transaction_id': transaction_id,
                    'credit_amt': '0',
                    'debit_amt': str(split_amount),
                    'balance': str(balance - split_amount * (i + 1)),
                    'date': transaction['date'],
                    'transaction_type': 'debit'
                }
                new_transactions.append(new_transaction)
                transaction_id += 1

            credit_transaction = {
               
                'transaction_id': transaction_id,
                'credit_amt': transaction['debit_amt'],
                'debit_amt': '0',
                'balance': transaction['balance'],
                'date': transaction['date'],
                'transaction_type': 'credit'
            }
            new_transactions.append(credit_transaction)
            transaction_id += 1
        else:
            transaction['transaction_id'] = transaction_id
            new_transactions.append(transaction)
            transaction_id += 1

    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(new_transactions)

# Example usage
input_file = 'transaction_list.csv'
output_file = 'split_transactions_new.csv'
split_debit_transactions(input_file, output_file)